# Generate App Hub's configuration

This notebook aim to produce a `config.yml` file to generate all the configuration needs to be set for deployment of App Hub on a remote cluster using Devops tools. 

This Notebook can be used for the following requirements:

* Initial dependencies
* Create the configuration for kubernetes' `PV`s, and `PVC`s(e.g. workspace volume, and calrissian volume)
* Generate the configuration for kubernetes' config maps
* Creation of different Profiles 


## Initial dependencies

In [1]:
import yaml
from apphub_configurator.models import *
from pathlib import Path
import os
import json
from pprint import pprint
current_dir = Path(os.getcwd())
parent_dir = current_dir.parent
current_dir

PosixPath('/home/p/Desktop/p/Terradue/EOEPCA/application-hub-context/config-generator/examples')

### Configuration



In [2]:
storage_class_rwo = "managed-nfs-storage"
storage_class_rwx = "managed-nfs-storage"

workspace_volume_size = "50Gi"
calrissian_volume_size = "50Gi"

## Volumes

In this section, the user will provide the data class objects for creation of volume:
- Volume for workspace
- Volume for calrissian
These two configuration will be use 

### Workspace Volume
In this section, the user will providing the configuration of a kubernetes Volume for development environment (i.e workspace). It is important to mention that this volume must keep data persistently. therefore, the user set `persist=True`

In [3]:
workspace_volume = Volume( # type: ignore
    name="workspace-volume",
    size=workspace_volume_size,
    claim_name="workspace-claim",
    mount_path="/workspace",
    storage_class=storage_class_rwo,
    access_modes=["ReadWriteOnce"],
    volume_mount=VolumeMount(name="workspace-volume", mount_path="/workspace"), # type: ignore
    persist=True,
)
workspace_volume

Volume(name='workspace-volume', claim_name='workspace-claim', size='50Gi', storage_class='managed-nfs-storage', access_modes=['ReadWriteOnce'], volume_mount=VolumeMount(name='workspace-volume', mount_path='/workspace'), persist=True)

### Calrissian Volume
In this section, the user will configure a **Kubernetes Volume** for Calrissian jobs. Since the job runs within Calrissian and does not require data retention on the Calrissian pod, therefore the user should set `persist=False`.  

In [4]:
calrissian_volume = Volume( # type: ignore
    name="calrissian-volume",
    claim_name="calrissian-claim",
    size=calrissian_volume_size,
    storage_class=storage_class_rwx,
    access_modes=["ReadWriteMany"],
    volume_mount=VolumeMount(name="calrissian-volume", mount_path="/calrissian"), # type: ignore
    persist=False,
)
calrissian_volume

Volume(name='calrissian-volume', claim_name='calrissian-claim', size='50Gi', storage_class='managed-nfs-storage', access_modes=['ReadWriteMany'], volume_mount=VolumeMount(name='calrissian-volume', mount_path='/calrissian'), persist=False)

## ConfigMaps

In this section, the user will provide the configuration for some kubernetes configmaps that is commonly used in this notebook including:
- bash login Configmap
- bash rc Configmap
- QGIS Configmap
- Stage-in/out Configmap
These configmaps will be mounted as files on different pods.  

### bash login

This configmap file aims to configure the Terminal across different profiles. 

In [5]:
bash_login_file_path = os.path.join(parent_dir, "config-maps/bash-login") 
with open(bash_login_file_path, "r") as f:
    content = f.read()

bash_login_cm = ConfigMap( # type: ignore
    name="bash-login",
    key="bash-login",
    content=content,
    readonly=True,
    persist=False,
    mount_path="/workspace/.bash_login",
)

### bash-rc 
This section explains how we configure a **Kubernetes ConfigMap** to manage the `.bashrc` file inside a pod. This config map will:
- It provides useful aliases and environment settings for users.
- It ensures a consistent environment across different pods.

In [6]:
bash_rc_cm_file_path = os.path.join(parent_dir, "config-maps/bash-rc") 
with open(bash_rc_cm_file_path, "r") as f:
    content = f.read()
bash_rc_cm = ConfigMap( # type: ignore
    name="bash-rc",
    key="bash-rc",
    content=content,
    readonly=True,
    persist=False,
    mount_path="/workspace/.bashrc",
)

### QGIS Configmap
The cell below will provide a Configmap configuration for QGIS setup.

In [7]:
init_qgis_file_path = os.path.join(parent_dir, "config-maps/init-qgis.sh") 
with open(init_qgis_file_path, "r") as f:
    content = f.read()

init_qgis_cm = ConfigMap( # type: ignore
    name="init",
    key="init",
    content=content,
    readonly=True,
    persist=False,
    mount_path="/opt/init/.init.sh",
)

### Stage-in/out Configmap
The cell below will provide a Configmap configuration for stage-in/out for [e-learning](#e-learning) profile.

In [8]:
init_stac_file_path = os.path.join(parent_dir, "config-maps/init-stac.sh")
with open(init_stac_file_path, "r") as f:
    content = f.read()

init_stac_cm = ConfigMap( # type: ignore
    name="init",
    key="init",
    content=content,
    readonly=True,
    persist=False,
    mount_path="/opt/init/.init.sh",
)

### Coder init Configmap
The cell below will provide a Configmap configuration for stage-in/out for [coder](#coder) profile.

In [9]:
init_cm_file_path = os.path.join(parent_dir, "config-maps/init.sh") 
with open(init_cm_file_path, "r") as f:
    content = f.read()

init_coder_cm = ConfigMap(
    name="init",
    key="init",
    content=content,
    readonly=True,
    persist=False,
    mount_path="/opt/init/.init.sh",
    default_mode="0660",
)

## Profiles
In the following section, the user will generate the configuration for different profiles on a remote cluster including:
- [Coder](#coder)
- [Jupyter Lab](#jupyterlab)
- [e-learning](#e-learning)
- [QGIS](#qgis)

In [10]:
profiles = []

### Coder

In [ ]:
coders = {
    "coder1": {
        "display_name": "Code Server Small",
        "slug": "ellip_studio_coder_slug_s",
        "cpu_limit": 2,
        "mem_limit": "8G",
    },
    "coder2": {
        "display_name": "Code Server Medium",
        "slug": "ellip_studio_coder_slug_m",
        "cpu_limit": 4,
        "mem_limit": "12G",
    },
}

for key, value in coders.items():
    coder_definition = ProfileDefinition( # type: ignore
        display_name=value["display_name"],
        slug=value["slug"],
        default=False,
        kubespawner_override=KubespawnerOverride( # type: ignore
            cpu_limit=value["cpu_limit"],
            mem_limit=value["mem_limit"],
            image="eoepca/pde-code-server:develop",
        ),
    )

    coder_profile = Profile( # type: ignore
        id=f"profile_studio_{key}",
        groups=["group-a", "group-b"],
        definition=coder_definition,
        node_selector={},
        volumes=[calrissian_volume, workspace_volume],
        config_maps=[
            bash_rc_cm,
        ],
        pod_env_vars={
            "HOME": "/workspace",
            "CONDA_ENVS_PATH": "/workspace/.envs",
        },
    )

    profiles.append(coder_profile)
profiles

[Profile(id='profile_studio_coder1', groups=['group-a', 'group-b'], definition=ProfileDefinition(display_name='Code Server Small', description=None, slug='ellip_studio_coder_slug_s', default=False, kubespawner_override=KubespawnerOverride(cpu_limit=2, cpu_guarantee=None, mem_limit='8G', mem_guarantee=None, image='eoepca/pde-code-server:develop', extra_resource_limits={}, extra_resource_guarantees={})), config_maps=[ConfigMap(name='bash-rc', key='bash-rc', mount_path='/workspace/.bashrc', default_mode=None, readonly=True, content='alias ll="ls -l"\nalias calrissian="/opt/conda/bin/calrissian --pod-nodeselectors /etc/calrissian/pod-node-selector.yml --stdout /calrissian/results.json --max-ram 16G --max-cores "8" --tmp-outdir-prefix /calrissian/tmp/ --outdir /calrissian/"\nalias cwltool="/opt/conda/bin/cwltool --podman"\n. /home/jovyan/.bashrc\n\n#alias aws="aws --endpoint-url=http://localstack:4566"\n\n# >>> conda initialize >>>\n# !! Contents within this block are managed by \'conda ini

### init.sh script

In [ ]:


init_context_volume_mount = InitContainerVolumeMount( # type: ignore
    mount_path="/opt/init/.init.sh", name="init", sub_path="init"
)
init_container = InitContainer( # type: ignore
    name="init-file-on-volume",
    image="eoepca/pde-code-server:develop",
    command=["sh", "-c", "sh /opt/init/.init.sh"],
    volume_mounts=[
        VolumeMount(name="workspace-volume", mount_path="/workspace"), # type: ignore
        init_context_volume_mount,
    ],
)

eoepca_demo_init_script_profile = Profile( # type: ignore
    id=f"profile_demo_init_script",
    groups=["group-a", "group-b"],
    definition=ProfileDefinition( # type: ignore
        display_name="Coder demo init script",
        description="This profile is used to demonstrate the use of an init script",
        slug="eoepca_demo_init_script",
        default=False,
        kubespawner_override=KubespawnerOverride( # type: ignore
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image="eoepca/pde-code-server:develop",
        ),
    ),
    node_selector={},
    volumes=[calrissian_volume, workspace_volume],
    config_maps=[init_coder_cm],
    pod_env_vars={
        "HOME": "/workspace",
        "CONDA_ENVS_PATH": "/workspace/.envs",
        "CONDARC": "/workspace/.condarc",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "CODE_SERVER_WS": "/workspace/mastering-app-package",
    },
    init_containers=[init_container],
)
profiles.append(eoepca_demo_init_script_profile)
profiles

[Profile(id='profile_studio_coder1', groups=['group-a', 'group-b'], definition=ProfileDefinition(display_name='Code Server Small', description=None, slug='ellip_studio_coder_slug_s', default=False, kubespawner_override=KubespawnerOverride(cpu_limit=2, cpu_guarantee=None, mem_limit='8G', mem_guarantee=None, image='eoepca/pde-code-server:develop', extra_resource_limits={}, extra_resource_guarantees={})), config_maps=[ConfigMap(name='bash-rc', key='bash-rc', mount_path='/workspace/.bashrc', default_mode=None, readonly=True, content='alias ll="ls -l"\nalias calrissian="/opt/conda/bin/calrissian --pod-nodeselectors /etc/calrissian/pod-node-selector.yml --stdout /calrissian/results.json --max-ram 16G --max-cores "8" --tmp-outdir-prefix /calrissian/tmp/ --outdir /calrissian/"\nalias cwltool="/opt/conda/bin/cwltool --podman"\n. /home/jovyan/.bashrc\n\n#alias aws="aws --endpoint-url=http://localstack:4566"\n\n# >>> conda initialize >>>\n# !! Contents within this block are managed by \'conda ini

### JupyterLab

In [ ]:
image = "jupyter/scipy-notebook"


eoepca_jupyter_lab_profile = Profile( # type: ignore
    id="profile_jupyter_lab",
    groups=["group-c"],
    definition=ProfileDefinition( # type: ignore
        display_name="Jupyter Lab",
        description="Jupyter Lab with Python 3.11",
        slug="eoepca_jupyter_lab",
        default=False,
        kubespawner_override=KubespawnerOverride( # type: ignore
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image=image,
        ),
    ),
    node_selector={},
    volumes=[workspace_volume],
    config_maps=[],
    pod_env_vars={
        "HOME": "/workspace",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "XDG_CONFIG_HOME": "/workspace/.config",
    },
)

profiles.append(eoepca_jupyter_lab_profile)

Image pull secret


In [ ]:
image_pull_secret = ImagePullSecret( # type: ignore
    name="cr-config",
    persist=False,
    data="ewogICAgImF1dGhzIjogewogICAgICAgICJjci50ZXJyYWR1ZS5jb20iOiB7CiAgICAgICAgICAgICJ1c2VybmFtZSI6ICJyb2JvdCRlb2VwY2EtcGx1cy1ybyIsCiAgICAgICAgICAgICJwYXNzd29yZCI6ICJQMlE4TnkyZ0lHODhkZkxveXlLN05QVUZVbHJOekFZSiIsCiAgICAgICAgICAgICJlbWFpbCI6ICJlb2VwY2EtcGx1c0B0ZXJyYWR1ZS5jb20iLAogICAgICAgICAgICAiYXV0aCI6ICJjbTlpYjNRa1pXOWxjR05oTFhCc2RYTXRjbTg2VURKUk9FNTVNbWRKUnpnNFpHWk1iM2w1U3pkT1VGVkdWV3h5VG5wQldVbz0iCiAgICAgICAgfQogICAgfQp9",
)

In [14]:
image = "cr.terradue.com/eoepca-plus/scipy-notebook@sha256:f339a9fa98d3d0c1fa8d7cc850e7f5a46845781f49bee86aacba059669d02d54"
image = "eoepca/iat-jupyterlab:develop"

eoepca_jupyter_lab_profile_2 = Profile(
    id="profile_jupyter_lab_2",
    groups=["group-c"],
    definition=ProfileDefinition(
        display_name="Jupyter Lab - profile 2",
        description="Jupyter Lab with Python 3.11 private image - demoes the use of an image pull secret",
        slug="eoepca_jupyter_lab_2",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image=image,
        ),
    ),
    node_selector={},
    volumes=[workspace_volume],
    config_maps=[],
    pod_env_vars={
        "HOME": "/workspace",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "XDG_CONFIG_HOME": "/workspace/.config",
    },
    image_pull_secrets=[image_pull_secret],
)

profiles.append(eoepca_jupyter_lab_profile_2)

In [16]:
localstack_manifest_path = os.path.join(parent_dir, "manifests/manifest.yaml") 
with open(localstack_manifest_path, "r") as f:
    content = yaml.safe_load_all(f.read())


localstack_manifest = Manifest(
    name="manifests",
    key="manifests",
    readonly=True,
    persist=False,
    content=[e for e in content],
)

### e-learning

In [ ]:
image = "docker.io/eoepca/pde-code-server@sha256:f57a3d5eabcae667e0db6e84a57b0c07c692c88f0fb5c8f6900ab8d5e38fcd40"

coder_profile_stac = Profile(
    id=f"profile_studio_coder_stac",
    groups=["group-a", "group-b"],
    definition=ProfileDefinition(
        display_name="Understanding STAC for input/output data modelling",
        description="Understand the role of STAC in input/output data manifests in EO data processing workflows",
        slug="eoepca_coder_slug_stac",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image=image,
        ),
    ),
    node_selector={},
    volumes=[workspace_volume],
    config_maps=[init_stac_cm, bash_rc_cm, bash_login_cm],
    pod_env_vars={
        "HOME": "/workspace",
        "CONDA_ENVS_PATH": "/workspace/.envs",
        "CONDARC": "/workspace/.condarc",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "XDG_CONFIG_HOME": "/workspace/.local",
        "XDG_DATA_HOME": "/workspace/.local/share/",
        "CWLTOOL_OPTIONS": "--podman",
        "CODE_SERVER_WS": "/workspace/stac-eoap",
        "AWS_DEFAULT_REGION": "us-east-1",
        "AWS_ACCESS_KEY_ID": "test",
        "AWS_SECRET_ACCESS_KEY": "test",
    },
    role_bindings=[],
    init_containers=[init_container],
    image_pull_secrets=[image_pull_secret],
    manifests=[localstack_manifest],
)

profiles.append(coder_profile_stac)

### QGIS

In [ ]:


image = "eoepca/iga-remote-desktop-qgis:1.1.3"

qgis_profile = Profile(
    id="profile_studio_desktop_qgis",
    groups=["group-a", "group-b"],
    definition=ProfileDefinition(
        display_name="QGIS on a Remote Desktop",
        description="Spatial visualization and decision-making tools for everyone",
        slug="eoepca_desktop_qgis",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_limit=2,
            mem_limit="2G",
            image=image,
        ),
    ),
    node_selector={},
    volumes=[workspace_volume],
    config_maps=[bash_rc_cm, bash_login_cm],
    pod_env_vars={"HOME": "/workspace"},
    default_url="desktop",
    init_containers=[],
)

profiles.append(qgis_profile)

## Write Configuration
The code below dump a `config.yaml` containing the information of App Hub.

In [19]:
config = Config(profiles=profiles)
eoepca_demo_config_path = str(Path(current_dir).parent.parent / 'eoepca-demo' / 'config.yml')
with open(eoepca_demo_config_path, "w") as file:
    yaml.dump(config.model_dump(), file)

In [20]:
# profiles